In [1]:
RUNS = [2]
LENGTHS = [2000, 10000, 20000, 30000]
DEPTHS = [3, 20, 40]



In [7]:
from string import Template
EXP_NAME_TEMP = Template("${DEPTH}d_${LENGTH_K}kl_${COUNT}")

JOB_FILE_TEMP = Template("""#!/bin/bash

#PBS -P ll21
#PBS -q gpuvolta
#PBS -l ncpus=12
#PBS -l ngpus=1
#PBS -l walltime=5:00:00
#PBS -l mem=12GB
#PBS -l wd
#PBS -l storage=gdata/ll21

./${EXP_NAME}.sh""")

SHELL_FILE_TEMP = Template("""#!/bin/bash
# Experiment Pipeline
# Robert Jeffrey <robert.jeffrey@anu.edu.au>

DEQ_DIR="/g/data1a/ll21/deq/DEQModel"

eval "$$(/home/659/rj4139/miniconda3/bin/conda shell.bash hook)"
conda activate honours

cd $${DEQ_DIR}
bash $${DEQ_DIR}/run_trellisnet_experiment.sh train --name ${EXP_NAME} --n_layer ${DEPTH} --pretrain_steps ${LENGTH} --seed ${COUNT}00 --timing --time_limit 16000 --force-deq-validation
""")

In [13]:
for run in RUNS:
    for length in LENGTHS:
        for depth in DEPTHS:
            exp_name = EXP_NAME_TEMP.substitute(DEPTH=depth, LENGTH_K=length//1000, COUNT=run)
            job_file = JOB_FILE_TEMP.substitute(EXP_NAME = exp_name)
            shell_file = SHELL_FILE_TEMP.substitute(EXP_NAME=exp_name, DEPTH=depth, LENGTH=length, COUNT=run)
            
            with open(exp_name+".job", 'w') as f:
                f.write(job_file)
            with open(exp_name+".sh", 'w') as f:
                f.write(shell_file)